In [1]:
import numpy as np
import time

# Define the number of VMs and their capabilities (100 and 1000 MIPS)
vm_capabilities = [100, 1000]
num_vms_per_edge_server = 10
num_edge_servers = 3  # Number of edge servers

# Define tasks
class Task:
    def __init__(self, id, length):
        self.id = id
        self.length = length

# Define MEC server with its VMs and processors
class MECServer:
    def __init__(self, id, num_vms, vm_capabilities):
        self.id = id
        self.vms = [{'capability': np.random.choice(vm_capabilities), 'processors': np.random.randint(1, 11)} for _ in range(num_vms)]
        print(f"MEC Server {self.id} VMs: {[vm['capability'] for vm in self.vms]} with processors: {[vm['processors'] for vm in self.vms]}")

# Define Cloud Server (for potential future use)
class CloudServer:
    def __init__(self, id, capability):
        self.id = id
        self.capability = capability
        print(f"Cloud Server {self.id} Capability: {self.capability}")

# Create Cloud Server
cloud_server = CloudServer(id=1, capability=10000)

# Create MEC servers
def create_edge_servers(num_edge_servers, num_vms_per_edge_server, vm_capabilities):
    return [MECServer(id=i, num_vms=num_vms_per_edge_server, vm_capabilities=vm_capabilities) for i in range(1, num_edge_servers + 1)]

# Function to calculate EPT for each task on each processor of a VM
def calculate_ept(task, vm_capability, num_processors):
    return task.length / (vm_capability * num_processors)

# Define the fitness function
def fitness_function(solution, ept_matrix, vm_capabilities):
    execution_time = np.max([np.sum(ept_matrix[:, j] * (solution == j)) for j in range(len(vm_capabilities))])
    energy_consumption = np.sum([np.sum(ept_matrix[:, j] * (solution == j)) * 0.65 for j in range(len(vm_capabilities))])
    b = 0.5  # Balancing parameter
    return (1 - b) * execution_time + b * energy_consumption

# Implement Improved Grey Wolf Optimizer (I-GWO)
def gwo_optimization(num_wolves, num_dimensions, max_iterations, fitness_function, ept_matrix, vm_capabilities):
    a = 2  # Coefficient

    # Initialize alpha, beta, and delta wolves
    alpha_wolf, beta_wolf, delta_wolf = np.zeros(num_dimensions), np.zeros(num_dimensions), np.zeros(num_dimensions)
    alpha_score, beta_score, delta_score = float('inf'), float('inf'), float('inf')

    wolves = np.random.randint(0, len(vm_capabilities), (num_wolves, num_dimensions))  # Initialize wolves

    for iteration in range(max_iterations):
        for i in range(num_wolves):
            fitness = fitness_function(wolves[i], ept_matrix, vm_capabilities)
            if fitness < alpha_score:
                alpha_score, alpha_wolf = fitness, wolves[i]
            elif fitness < beta_score:
                beta_score, beta_wolf = fitness, wolves[i]
            elif fitness < delta_score:
                delta_score, delta_wolf = fitness, wolves[i]

        a = 2 - iteration * (2 / max_iterations)  # Update the coefficient 'a'

        for i in range(num_wolves):
            for j in range(num_dimensions):
                r1, r2 = np.random.rand(), np.random.rand()
                A1, C1 = 2 * a * r1 - a, 2 * r2

                D_alpha = abs(C1 * alpha_wolf[j] - wolves[i][j])
                X1 = alpha_wolf[j] - A1 * D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2, C2 = 2 * a * r1 - a, 2 * r2

                D_beta = abs(C2 * beta_wolf[j] - wolves[i][j])
                X2 = beta_wolf[j] - A2 * D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3, C3 = 2 * a * r1 - a, 2 * r2

                D_delta = abs(C3 * delta_wolf[j] - wolves[i][j])
                X3 = delta_wolf[j] - A3 * D_delta

                wolves[i][j] = (X1 + X2 + X3) / 3

    return alpha_wolf, alpha_score

# Implement Dimension Learning-based Hunting (DLH) Strategy
def dimension_learning_hunting(wolves, num_wolves, num_dimensions, alpha_wolf, fitness_function, ept_matrix, vm_capabilities):
    new_wolves = np.copy(wolves)

    for i in range(num_wolves):
        RS_i = np.linalg.norm(wolves[i] - alpha_wolf)
        neighbors = []

        for j in range(num_wolves):
            if np.linalg.norm(wolves[i] - wolves[j]) < RS_i:
                neighbors.append(wolves[j])

        if len(neighbors) > 0:
            random_neighbor = neighbors[np.random.randint(len(neighbors))]
            new_position = wolves[i] + np.random.rand() * (random_neighbor - wolves[i])
            new_fitness = fitness_function(new_position, ept_matrix, vm_capabilities)

            if new_fitness < fitness_function(wolves[i], ept_matrix, vm_capabilities):
                new_wolves[i] = new_position

    return new_wolves

# Task scheduling optimization
def task_scheduling(num_wolves, num_dimensions, max_iterations, ept_matrix, vm_capabilities):
    # Initialize population
    wolves = np.random.randint(0, len(vm_capabilities), (num_wolves, num_dimensions))

    alpha_wolf, alpha_score = gwo_optimization(num_wolves, num_dimensions, max_iterations, fitness_function, ept_matrix, vm_capabilities)

    for iteration in range(max_iterations):
        wolves = dimension_learning_hunting(wolves, num_wolves, num_dimensions, alpha_wolf, fitness_function, ept_matrix, vm_capabilities)
        alpha_wolf, alpha_score = gwo_optimization(num_wolves, num_dimensions, max_iterations, fitness_function, ept_matrix, vm_capabilities)

    return alpha_wolf, alpha_score

# Define the different numbers of tasks to be tested
num_tasks_list = [40, 80, 120, 160]

# Maximum iterations
max_iterations = 15

# Metrics storage
execution_times = []
energy_consumptions = []
energy_savings = []

# Run the task scheduling optimization for different numbers of tasks
for num_tasks in num_tasks_list:
    start_time = time.time()
    
    task_lengths = np.random.randint(10, 510, size=num_tasks)
    tasks = [Task(id=i, length=task_lengths[i]) for i in range(num_tasks)]
    print(f"Tasks Lengths: {[task.length for task in tasks]}")
    
    edge_servers = create_edge_servers(num_edge_servers, num_vms_per_edge_server, vm_capabilities)
    total_vms = sum(vm['processors'] for server in edge_servers for vm in server.vms)
    ept_matrix = np.zeros((len(tasks), total_vms))
    vm_capabilities_flat = np.concatenate([[vm['capability']] * vm['processors'] for server in edge_servers for vm in server.vms])

    for i, task in enumerate(tasks):
        for j, vm in enumerate(vm_capabilities_flat):
            ept_matrix[i][j] = calculate_ept(task, vm, vm_capabilities_flat[j])

    best_wolf, best_fitness = task_scheduling(num_wolves=30, num_dimensions=num_tasks, max_iterations=max_iterations, ept_matrix=ept_matrix, vm_capabilities=vm_capabilities_flat)
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    # Calculate energy consumption and savings
    energy_consumption = np.sum([np.sum(ept_matrix[:, j] * (best_wolf == j)) * 0.65 for j in range(len(vm_capabilities_flat))])
    energy_saving = (np.sum(task_lengths) - energy_consumption) / np.sum(task_lengths) * 100
    
    execution_times.append(execution_time)
    energy_consumptions.append(energy_consumption)
    energy_savings.append(energy_saving)
    
    print(f"Number of Tasks: {num_tasks}")
    print(f"Best Wolf: {best_wolf}")
    print(f"Best Fitness: {best_fitness}")
    print(f"Execution Time: {execution_time:.2f} seconds")
    print(f"Energy Consumption: {energy_consumption:.2f}")
    print(f"Energy Savings: {energy_saving:.2f}%\n")

# Impact of the number of VMs on task completion time
num_vms_list = [10, 15, 20, 25, 30]
task_completion_times = []

for num_vms in num_vms_list:
    start_time = time.time()
    
    num_tasks = 40
    task_lengths = np.random.randint(10, 510, size=num_tasks)
    tasks = [Task(id=i, length=task_lengths[i]) for i in range(num_tasks)]
    print(f"Tasks Lengths for VMs {num_vms}: {[task.length for task in tasks]}")
    
    edge_servers = create_edge_servers(num_edge_servers, num_vms, vm_capabilities)
    total_vms = sum(vm['processors'] for server in edge_servers for vm in server.vms)
    ept_matrix = np.zeros((len(tasks), total_vms))
    vm_capabilities_flat = np.concatenate([[vm['capability']] * vm['processors'] for server in edge_servers for vm in server.vms])

    for i, task in enumerate(tasks):
        for j, vm in enumerate(vm_capabilities_flat):
            ept_matrix[i][j] = calculate_ept(task, vm, vm_capabilities_flat[j])

    best_wolf, best_fitness = task_scheduling(num_wolves=30, num_dimensions=num_tasks, max_iterations=max_iterations, ept_matrix=ept_matrix, vm_capabilities=vm_capabilities_flat)
    
    end_time = time.time()
    completion_time = end_time - start_time
    task_completion_times.append(completion_time)
    
    print(f"Number of VMs: {num_vms}")
    print(f"Task Completion Time: {completion_time:.2f} seconds\n")

# Print all collected metrics
print("\nExecution Times for Different Numbers of Tasks:", execution_times)
print("Energy Consumptions for Different Numbers of Tasks:", energy_consumptions)
print("Energy Savings for Different Numbers of Tasks:", energy_savings)
print("Task Completion Times for Different Numbers of VMs (Tasks = 40):", task_completion_times)


Cloud Server 1 Capability: 10000
Tasks Lengths: [215, 126, 274, 486, 116, 37, 302, 428, 313, 178, 237, 295, 316, 395, 312, 83, 458, 424, 373, 439, 37, 167, 70, 45, 118, 103, 478, 218, 222, 148, 285, 380, 24, 229, 272, 310, 76, 482, 428, 15]
MEC Server 1 VMs: [100, 100, 100, 1000, 100, 100, 1000, 100, 1000, 100] with processors: [1, 4, 10, 4, 1, 8, 10, 3, 10, 1]
MEC Server 2 VMs: [100, 100, 1000, 1000, 100, 1000, 1000, 100, 100, 1000] with processors: [4, 9, 8, 7, 2, 8, 7, 10, 2, 6]
MEC Server 3 VMs: [1000, 100, 1000, 100, 1000, 1000, 100, 1000, 100, 1000] with processors: [8, 6, 2, 7, 5, 10, 2, 1, 2, 9]
Number of Tasks: 40
Best Wolf: [ 21  61  49 197   0 246 137  44 126  74  31  78  55   0 -10   9  55 217
   0  60   0  12  18 141 329  29  14 332 390  16   5 209   7  35  84 103
 187  48  32   0]
Best Fitness: 0.07054975000000001
Execution Time: 31.09 seconds
Energy Consumption: 0.29
Energy Savings: 100.00%

Tasks Lengths: [18, 496, 317, 410, 475, 11, 458, 325, 502, 236, 98, 96, 432, 337